In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import datetime

In [2]:
bertcrf = pd.read_csv("compare/BERT_CRF_pred.csv",encoding="utf-8")
bert = pd.read_csv("compare/BERT_pred.csv",encoding="utf-8")
bertcrf.head()

textid  sentid                     text  answer  pred
0      11       0            骗子是广西人士真名叫刘衍林     701     7
1      11       1  专门以发布人家二手车的图片或者人家私家车的图片     102     1
2      11       2      来冒充自己是二手车卖家来招摇撞骗的手段     102     2
3      11       3                   专门骗取押金     305     3
4      11       4                  当你付给他押金     701     7

In [13]:
res = pd.read_csv("compare/预测比较.csv",encoding="utf-8",engine='python')
res

类别  实际该类别个数  BERT预测该类别个数  序列化模型预测该类别个数  BERT预测正确的个数  序列化模型预测正确的个数  \
0   101     1685         1649          1696         1423          1464   
1   102      505          522           509          377           375   
2   103      487          476           492          394           400   
3   104      292          285           269          220           207   
4   105      288          262           287          213           219   
5   106        6            2             0            0             0   
6   107      683          724           767          351           362   
7   201      131          117           154           98           111   
8   202       15           12             0            7             0   
9   203       15           16             0            7             0   
10  204        2            1             0            1             0   
11  205      112          129           130           98            97   
12  206       26           21             0            7             0   
13  301       22            1             0            1             0   
14  302       70           47             0           29             0   
15  303       24           10             0            2             0   
16  304      753          795           769          630           624   
17  305      493          473           412          270           253   
18  306      205           44             0           17             0   
19  307      304          305           313          201           199   
20  308       44            1             0            0             0   
21  401       62           68            64           46            46   
22  402      615          647           682          547           555   
23  403       17           16             0            9             0   
24  404       60           22             0           20             0   
25  405       28           29             0           21             0   
26  406       27            6             0            4             0   
27  407        0            0             0            0             0   
28  408       19            8             0            3             0   
29  501     1402         1474          1434         1361          1349   
30  502      154          116           140           90            96   
31  503      341          400           390          202           206   
32  504     1585         1672          1631         1323          1322   
33  505       25            1             0            0             0   
34  601      106           95           101           79            74   
35  602       82           83            91           64            61   
36  603       60           74             0           46             0   
37  701     8930         9072          9344         7702          7845   

    Recall(BERT)  Recall(序列化模型)  Precision(BERT)  Precision(序列化模型)  F1(BERT)  \
0         0.8445         0.8688           0.8629            0.8632    0.8536   
1         0.7465         0.7426           0.7222            0.7367    0.7342   
2         0.8090         0.8214           0.8277            0.8130    0.8183   
3         0.7534         0.7089           0.7719            0.7695    0.7626   
4         0.7396         0.7604           0.8130            0.7631    0.7745   
5         0.0000         0.0000           0.0000               NaN       NaN   
6         0.5139         0.5300           0.4848            0.4720    0.4989   
7         0.7481         0.8473           0.8376            0.7208    0.7903   
8         0.4667         0.0000           0.5833               NaN    0.5185   
9         0.4667         0.0000           0.4375               NaN    0.4516   
10        0.5000         0.0000           1.0000               NaN    0.6667   
11        0.8750         0.8661           0.7597            0.7462    0.8133   
12        0.2692         0.0000           0.3333               NaN    0.2979  

In [3]:
bert.head()

textid  sentid                     text  answer  pred
0      11       0            骗子是广西人士真名叫刘衍林     701     7
1      11       1  专门以发布人家二手车的图片或者人家私家车的图片     102     1
2      11       2      来冒充自己是二手车卖家来招摇撞骗的手段     102     2
3      11       3                   专门骗取押金     305     3
4      11       4                  当你付给他押金     701     7

In [8]:
bertcrf['crf_label'] = bertcrf['answer'].apply(lambda x: int(str(x)[0]))
bert['bert_label'] = bert['answer'].apply(lambda x: int(str(x)[0]))
crf_wrong = bertcrf[bertcrf['crf_label']!=bertcrf['pred']]
bert_wrong = bert[bert['bert_label']!=bert['pred']]
crf_wrong.head()

textid  sentid                 text  answer  pred  crf_label
2       11       2  来冒充自己是二手车卖家来招摇撞骗的手段     102     2          1
8       11       8               以免祸害人间     701     6          7
9       15       0         刘华俊一直打电话催促付款     701     4          7
11      15       2       他给我打电话让帮忙点击下就行     701     4          7
13      17       0      对方先是说188能达到我的需求     104     7          1

In [9]:
bert_wrong.head()

textid  sentid                 text  answer  pred  bert_label
2       11       2  来冒充自己是二手车卖家来招摇撞骗的手段     102     2           1
8       11       8               以免祸害人间     701     6           7
9       15       0         刘华俊一直打电话催促付款     701     4           7
11      15       2       他给我打电话让帮忙点击下就行     701     4           7
13      17       0      对方先是说188能达到我的需求     104     7           1

In [10]:
wrong_total = pd.merge(bert_wrong,crf_wrong,on=['textid','sentid','text'],how='outer')
wrong_total

textid  sentid                                               text  \
0         11       2                                来冒充自己是二手车卖家来招摇撞骗的手段   
1         11       8                                             以免祸害人间   
2         15       0                                       刘华俊一直打电话催促付款   
3         15       2                                     他给我打电话让帮忙点击下就行   
4         17       0                                    对方先是说188能达到我的需求   
5         17       2                                              又要184   
6         36       2                                              说不用垫付   
7         36       4                                   这个后来她又给我推荐商家的微信号   
8         36       8                               商家发了个二维码让我支付宝扫码1850元   
9         36      10                                        后来又让我支付927元   
10        36      14                                          我那时就知道被骗了   
11        36      15                                           他说给我申请退款   
12        36      17                                        现在已经超过12小时了   
13        36      23                                          我那时就知道被骗了   
14        36      24                                           他说给我申请退款   
15        36      26                                        现在已经超过12小时了   
16        44       7                                     以借款贷款的名义让我支付给他   
17        50       0                                             对方自报商家   
18        50       3                                            发了个链接说是   
19        55       0                           首先让我在淘宝里家具店铺浏览后让对一个沙发下单后   
20        55       1                                        发一个链接让浏览器打开   
21        77       1                                    刷单洗钱微信支付宝共1万五千多   
22        96       7                                              只需借出来   
23        96       8                                             然后系统还款   
24        96       9                                              不需要利息   
25        96      10                                      然后他登我支付宝号进行转款   
26        97       1                                            我付了兼职的钱   
27        97       2                                               弄了兼职   
28       106       0                                             我买了个手机   
29       106       4                                     又说不够3500办不了退款单   
...      ...     ...                                                ...   
1109    6435       6                           之后又退回800元回我卡上就叫我联系QQ工作人员   
1110    6625       5                                         显示不足后用花呗支付   
1111    6669       0          对方说支付完成后给予退还但是又说订单不能退又让充276元我说充不了又让借我说借不了   
1112    7123       1                                          结果按她说的垫付了   
1113    7314       0                                           发来的东西不一样   
1114    7457       2                                               不要押金   
1115    7533       1  100元等等任务是看支付宝信用度给你派发的而不是人人都可以做必须成年才可以做的一个任务10分...   
1116    7685       0                  他发给我一张二维码图片让我扫付款然后让我用一张余额不足的银行卡支付   
1117    7788       0  让我用花呗付款说是有验证码付完了没有然后说给我退款也没退然后隔天又让我用余额宝付款完事给我发...   
1118    7808       1                                                送双鞋   
1119    8082       2                                我先他说的做完后就在我花呗扣了765元   
1120    8173       4                                     他说验证码一分钟之后就没用了   
1121    8609       1                                              会给我发货   
1122    8624       0                   在支付宝中可以代缴罚款但是又转到第三方付了款又显示不出来我的订单   
1123    8699       7                                        他也对我说一定退钱给我   
1124    8843       1                                         说第二天一早给我退钱   
1125    9115       2        刷了三单就不敢在刷了我说我的钱怎么办他说商家会退我等一天了也没退我再联系就联系不到人了   
1126    9167       0                                                买东西   
1127    9579      11                                               帮帮我啊   
1128    9866       6                              然后要给我退款问我交易截图和支付宝姓名账号   
1129    9870       2                                      结果我在输入密码后付款成功   
1130    9896       5           

In [11]:
del wrong_total['answer_y']
wrong_total.rename(columns={'answer_x':'answer','pred_x':'bert_pred','answer_y':'crf_answer','pred_y':'crf_pred'},inplace=True) 
wrong_total

textid  sentid                                               text  \
0         11       2                                来冒充自己是二手车卖家来招摇撞骗的手段   
1         11       8                                             以免祸害人间   
2         15       0                                       刘华俊一直打电话催促付款   
3         15       2                                     他给我打电话让帮忙点击下就行   
4         17       0                                    对方先是说188能达到我的需求   
5         17       2                                              又要184   
6         36       2                                              说不用垫付   
7         36       4                                   这个后来她又给我推荐商家的微信号   
8         36       8                               商家发了个二维码让我支付宝扫码1850元   
9         36      10                                        后来又让我支付927元   
10        36      14                                          我那时就知道被骗了   
11        36      15                                           他说给我申请退款   
12        36      17                                        现在已经超过12小时了   
13        36      23                                          我那时就知道被骗了   
14        36      24                                           他说给我申请退款   
15        36      26                                        现在已经超过12小时了   
16        44       7                                     以借款贷款的名义让我支付给他   
17        50       0                                             对方自报商家   
18        50       3                                            发了个链接说是   
19        55       0                           首先让我在淘宝里家具店铺浏览后让对一个沙发下单后   
20        55       1                                        发一个链接让浏览器打开   
21        77       1                                    刷单洗钱微信支付宝共1万五千多   
22        96       7                                              只需借出来   
23        96       8                                             然后系统还款   
24        96       9                                              不需要利息   
25        96      10                                      然后他登我支付宝号进行转款   
26        97       1                                            我付了兼职的钱   
27        97       2                                               弄了兼职   
28       106       0                                             我买了个手机   
29       106       4                                     又说不够3500办不了退款单   
...      ...     ...                                                ...   
1109    6435       6                           之后又退回800元回我卡上就叫我联系QQ工作人员   
1110    6625       5                                         显示不足后用花呗支付   
1111    6669       0          对方说支付完成后给予退还但是又说订单不能退又让充276元我说充不了又让借我说借不了   
1112    7123       1                                          结果按她说的垫付了   
1113    7314       0                                           发来的东西不一样   
1114    7457       2                                               不要押金   
1115    7533       1  100元等等任务是看支付宝信用度给你派发的而不是人人都可以做必须成年才可以做的一个任务10分...   
1116    7685       0                  他发给我一张二维码图片让我扫付款然后让我用一张余额不足的银行卡支付   
1117    7788       0  让我用花呗付款说是有验证码付完了没有然后说给我退款也没退然后隔天又让我用余额宝付款完事给我发...   
1118    7808       1                                                送双鞋   
1119    8082       2                                我先他说的做完后就在我花呗扣了765元   
1120    8173       4                                     他说验证码一分钟之后就没用了   
1121    8609       1                                              会给我发货   
1122    8624       0                   在支付宝中可以代缴罚款但是又转到第三方付了款又显示不出来我的订单   
1123    8699       7                                        他也对我说一定退钱给我   
1124    8843       1                                         说第二天一早给我退钱   
1125    9115       2        刷了三单就不敢在刷了我说我的钱怎么办他说商家会退我等一天了也没退我再联系就联系不到人了   
1126    9167       0                                                买东西   
1127    9579      11                                               帮帮我啊   
1128    9866       6                              然后要给我退款问我交易截图和支付宝姓名账号   
1129    9870       2                                      结果我在输入密码后付款成功   
1130    9896       5           

In [ ]:
diff_wrong